In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('dark_background')
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Conv1D, GlobalMaxPooling1D, SpatialDropout1D

Using TensorFlow backend.


In [2]:
df_train = pd.read_csv('train.tsv', sep='\t')

print('train set: {0}'.format(df_train.shape))
df_train.head(10)

train set: (156060, 4)


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
5,6,1,of escapades demonstrating the adage that what...,2
6,7,1,of,2
7,8,1,escapades demonstrating the adage that what is...,2
8,9,1,escapades,2
9,10,1,demonstrating the adage that what is good for ...,2


In [3]:
df_test = pd.read_csv('test.tsv', sep='\t')

print('test set: {0}'.format(df_train.shape))
df_test.head(10)

test set: (156060, 4)


,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine
5,156066,8545,intermittently pleasing but
6,156067,8545,intermittently pleasing
7,156068,8545,intermittently
8,156069,8545,pleasing
9,156070,8545,but


In [11]:
replace_list = {r"i'm": 'i am',
                r"'re": ' are',
                r"let’s": 'let us',
                r"'s":  ' is',
                r"'ve": ' have',
                r"can't": 'can not',
                r"cannot": 'can not',
                r"shan’t": 'shall not',
                r"n't": ' not',
                r"'d": ' would',
                r"'ll": ' will',
                r"'scuse": 'excuse',
                ',': ' ,',
                '.': ' .',
                '!': ' !',
                '?': ' ?',
                '\s+': ' '}

def clean_text(text):
    text = text.lower()
    for s in replace_list:
        text = text.replace(s, replace_list[s])
    text = ' '.join(text.split())
    return text

X_train = df_train['Phrase'].apply(lambda p: clean_text(p))
phrase_len = X_train.apply(lambda p: len(p.split(' ')))
max_phrase_len = phrase_len.max() + 10
print('max phrase len: {0}'.format(max_phrase_len))

plt.figure(figsize = (10, 8))
plt.hist(phrase_len, alpha = 0.2, density = True)
plt.xlabel('phrase len')
plt.ylabel('probability')
plt.grid(alpha = 0.25)

KeyError: 'Sentiment'

In [5]:
batch_size = 512
epochs = 8
num_classes = 5
max_words = 8192

In [6]:
y_train = df_train['Sentiment']

tokenizer = Tokenizer(
    num_words = max_words,
    filters = '"#$%&()*+-/:;<=>@[\]^_`{|}~'
)

tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen = max_phrase_len)
y_train = to_categorical(y_train.values)

# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1)
# print('X_train size: {0}, '.format(X_train.shape) +
#       'y_train size: {0}\n'.format(y_train.shape) +
#       'X_val size: {0}, '.format(X_val.shape) +
#       'y_val size: {0}'.format(y_val.shape))

In [7]:
model_lstm = Sequential()
model_lstm.add(Embedding(max_words, 256))
model_lstm.add(SpatialDropout1D(0.3))
model_lstm.add(LSTM(256, dropout=0.3, recurrent_dropout=0.3))
model_lstm.add(Dense(256, activation='relu'))
model_lstm.add(Dropout(0.3))
model_lstm.add(Dense(num_classes, activation='softmax'))
model_lstm.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [8]:
history = model_lstm.fit(
    X_train,
    y_train,
#     validation_data = (X_val, y_val),
    validation_split = 0.1,
    epochs = 8,
    batch_size = 512,
    verbose = 1
)

Train on 140454 samples, validate on 15606 samples
Epoch 1/8
140454/140454 [==============================] - 141s 1ms/step - loss: 1.0533 - acc: 0.5802 - val_loss: 1.0071 - val_acc: 0.5929
Epoch 2/8
140454/140454 [==============================] - 139s 992us/step - loss: 0.8540 - acc: 0.6570 - val_loss: 1.0072 - val_acc: 0.6019
Epoch 3/8
140454/140454 [==============================] - 139s 993us/step - loss: 0.8029 - acc: 0.6758 - val_loss: 1.0017 - val_acc: 0.6046
Epoch 4/8
140454/140454 [==============================] - 139s 992us/step - loss: 0.7676 - acc: 0.6880 - val_loss: 1.0113 - val_acc: 0.6075
Epoch 5/8
140454/140454 [==============================] - 139s 992us/step - loss: 0.7432 - acc: 0.6967 - val_loss: 1.0345 - val_acc: 0.6042
Epoch 6/8
140454/140454 [==============================] - 139s 993us/step - loss: 0.7226 - acc: 0.7038 - val_loss: 1.0693 - val_acc: 0.6109
Epoch 7/8
140454/140454 [==============================] - 139s 993us/step - loss: 0.7045 - acc: 0.7110 -

In [12]:
plt.clf()
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

NameError: name 'y_test' is not defined

In [ ]:
plt.clf()
acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()